### Twitter's Trending Topics

In [ ]:
import pandas as pd

In [ ]:
path = '/class/itpmssd/datasets/tts_nov_2015.csv'
df = pd.read_csv(path, header=0, names=["id","name","woeid","location","datetime"], index_col='datetime', parse_dates=True)
df = df.drop(['id','woeid'], 1) # 1 -> axis = 1

In [ ]:
# count all trending topics
df.groupby('name').size()

In [ ]:
# show top trends for a geographic location
df.groupby(['location','name']).size()['New York'].order(ascending=False)

In [ ]:
# count top trends in terms of # of appearances
df.groupby('name').size().order(ascending=False)[:20]

# Paris

In [ ]:
paris = df[df.name=='Paris']
paris.head()

In [ ]:
paris.location.unique()

In [ ]:
paris.groupby('location').size().order(ascending=False)[:20]

In [ ]:
top10_paris_locs = paris.groupby('location').size().order(ascending=False)[:10]

In [ ]:
top10_paris_locs

In [ ]:
new_df = paris[[x in top10_paris_locs.index for x in paris.location]]
new_df

In [ ]:
grouped = new_df.groupby([new_df.index.date, new_df.location]).agg(['count'])
grouped

In [ ]:
for loc in top10_paris_locs.index:
    cur_df = paris[paris['location']==loc]
    cur_df_ts = cur_df.groupby(cur_df.index.date).count()['location']
    cur_df_ts.plot(kind="line",rot=25, label=loc, figsize=(14, 8), legend=True)
    
plt.title('top 10 cities where Paris trended')
plt.ylabel('aggregate number of 5 minute intervals')
plt.xlabel('date')

In [ ]:
all_paris_locs = paris.groupby('location').size().order(ascending=False)

for loc in all_paris_locs.index:
    cur_df = paris[paris['location']==loc]
    cur_df_ts = cur_df.groupby(cur_df.index.date).count()['location'].reindex(wanted_range).fillna(0)
    cur_df_ts.plot(kind="line",rot=25, label=loc, color='black', figsize=(14, 8), legend=False, alpha=.1)
    
plt.title('Paris - all cities')
plt.ylabel('aggregate number of 5 minute intervals')
plt.xlabel('date')

# Ronaldo vs. Messi

In [ ]:
ronaldo = df[df['name']=='Ronaldo']
ronaldo.head()

In [ ]:
g_ronaldo = ronaldo.groupby([ronaldo.index.date]).size()
g_ronaldo

In [ ]:
%pylab inline

g_ronaldo.plot(figsize=(12,6))

In [ ]:
# we see a standard deviation of 525.85 in this case - for Ronaldo in the given period

g_ronaldo.describe()

In [ ]:
import numpy as np

arr = np.array(g_ronaldo)
arr

In [ ]:
# in order to identify spikes, we look at the derivative function -> differences between adjacent items, measuring diffs, changes

np.diff(arr)

In [ ]:
messi = df[df['name']=='Messi']
g_messi = messi.groupby([messi.index.date]).size()

In [ ]:
new_pd = pd.concat([g_messi, g_ronaldo], axis=1)
new_pd = pd.DataFrame({'Messi': g_messi,'Ronaldo':g_ronaldo})

In [ ]:
wanted_range = pd.date_range(start=min(new_pd.index),end=max(new_pd.index))
new_pd = new_pd.reindex(wanted_range).fillna(0)

In [ ]:
new_pd.plot(rot=25, figsize=(12,6))

In [ ]:
new_pd.plot(kind="bar", rot=25, figsize=(22,8))

In [ ]:
new_pd.Messi.plot(figsize=(12,6))

### Top trends - correlate?

In [ ]:
# top twitter trends --> unicode here important
#top = [x.decode('utf-8') for x in df.groupby('name').size().order(ascending=False)[:4].keys()]
top = [x for x in df.groupby('name').size().order(ascending=False)[:10].keys()]

In [ ]:
top

In [ ]:
top_pd = df[[x in top for x in df.name]]

In [ ]:
g_top = top_pd.groupby(['name',top_pd.index.date]).size()

In [ ]:
final_pd = pd.DataFrame()

for trend in top:
    final_pd[trend]=g_top[trend].reindex(wanted_range).fillna(0)

In [ ]:
final_pd.plot(figsize=(18,6))

In [ ]:
# correlation matrix
sns.corrplot(final_pd)